# Data Loader, Model, Trainning
# Dependencies

In [1]:
%load_ext autoreload

In [2]:
autoreload 2

In [3]:
from preprocess import *
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

I0706 08:57:47.469936 139908435564352 file_utils.py:39] PyTorch version 1.5.1 available.
I0706 08:57:49.104424 139908435564352 tokenization_utils_base.py:1254] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00


# Mode

In [25]:
TEST = True
LOG = True

# Hyper Params

In [4]:
BATCH_SIZE = 64

# File Path ...

In [5]:
train_data_path = './FGC_release_1.7.13/FGC_release_all_dev.json'

In [7]:
#default tolkenizer is used
train_set = FGC_Dataset(train_data_path, "train") 

In [18]:
print(train_set.tokenizer.convert_ids_to_tokens(train_set[0][0]), "\n",
train_set[0][1],"\n",
train_set[0][2])

['[CLS]', '苏', '东', '坡', '在', '中', '国', '历', '史', '上', '，', '是', '哪', '一', '个', '朝', '代', '的', '人', '？', '[SEP]', '苏', '轼', '（', '103', '##7', '年', '1', '月', '8', '日', '－', '110', '##1', '年', '8', '月', '24', '日', '）', '，', '[SEP]'] 
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) 
 tensor(True)


In [22]:
"""
DataLoader for minibatch
in each batch
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # use(have) label or not
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
    
    # zero pad to same length
    tokens_tensors = pad_sequence(tokens_tensors,  batch_first=True)
    segments_tensors = pad_sequence(segments_tensors,  batch_first=True)
    
    # attention masks, set none-padding part to 1 for LM to attend
    masks_tensors = torch.zeros(tokens_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill( tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids


# DataLoader
# collate_fn : list of instances to minibatch
trainloader = DataLoader(train_set, batch_size=BATCH_SIZE, collate_fn=create_mini_batch)

In [27]:
if TEST:
    data = next(iter(trainloader))

    tokens_tensors, segments_tensors, masks_tensors, label_ids = data

    print(f"""
    tokens_tensors.shape   = {tokens_tensors.shape} 
    {tokens_tensors}
    ------------------------
    segments_tensors.shape = {segments_tensors.shape}
    {segments_tensors}
    ------------------------
    masks_tensors.shape    = {masks_tensors.shape}
    {masks_tensors}
    ------------------------
    label_ids.shape        = {label_ids.shape}
    {label_ids}
    """)


    tokens_tensors.shape   = torch.Size([64, 55]) 
    tensor([[ 101, 5722,  691,  ...,    0,    0,    0],
        [ 101, 5722,  691,  ...,    0,    0,    0],
        [ 101, 5722,  691,  ...,    0,    0,    0],
        ...,
        [ 101, 5722,  691,  ...,    0,    0,    0],
        [ 101, 5722,  691,  ...,    0,    0,    0],
        [ 101, 5722,  691,  ...,    0,    0,    0]])
    ------------------------
    segments_tensors.shape = torch.Size([64, 76])
    tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
    ------------------------
    masks_tensors.shape    = torch.Size([64, 55])
    tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
    ------------